# Quantum Key Distribution

Quantum key distribution is the process of distributing cryptographic keys between parties using quantum methods. Due to the unique properties of quantum information compared to classical, the security of a key can be guarunteed (as any unwelcomed measurement would change the state of quantum information transmitted).

In this file, we see the use of SeQUeNCe to simulate quantum key distribution between two adjacent nodes. The first example demonstrates key distribution alone (using the BB84 protocol), while the second example demonstrates additional error correction with the cascade protocol. The network topology, including hardware components, is shown below:

<img src="./notebook_images/QKD_topo.png" width="500"/>

## Example 1: Only BB84

### Import

We must first import the necessary tools from SeQUeNCe to run our simulations.

- `Timeline` is the main simulation tool, providing an interface for the discrete-event simulation kernel.
- `QKDNode` provides a ready-to-use quantum node for quantum key distribution, including necessary hardware and protocol implementations.
- `QuantumChannel` and `ClassicalChannel` are communication links between quantum nodes, providing models of optical fibers.
- The `pair_bb84_protocols` function is used to explicitly pair 2 node instances for key distribution, and establishes one node as the sender "Alice" and one as the receiver "Bob".

In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.qkd.BB84 import pair_bb84_protocols

### Control and Collecting Metrics

Several elements of SeQUeNCe automatically collect simple metrics. This includes the BB84 protocol implementation, which collects key error rates, throughput, and latency. For custom or more advanced metrics, custom code may need to be written and applied. See the documentation for a list of metrics provided by default for each simulation tool.

Here, we create a `KeyManager` class to collect a custom metric (in this case, simply collect all of the generated keys and their generation time) and to provide an interface for the BB84 Protocol. To achieve this, we use the `push` and `pop` functions provided by the protocol stack on QKD nodes. `push` is used to send information down the stack (from the key manager to BB84 in this example) while `pop` is used to send information upwards (from BB84 to the key manager). Different protocols may use these interfaces for different data but only BB84 is shown in this example.

In [3]:
class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) # interface for BB84 to generate key
            
    def pop(self, info): # interface for BB84 to return generated keys
        self.keys.append(info)
        self.times.append(self.timeline.now() * 1e-9)

### Building the Simulation

We are now ready to build the simulation itself. This example follows the usual process to ensure that all tools function properly:

1. Create the timeline for the simulation
2. Create the simulated network topology (here this is done explicitly, but this may also be handled by functions of the `Topology` class under `sequence.topology.topology`)
3. Instantiate custom protocols and ensure all protocols are set up (paired) properly (if necessary)
4. Initialize and run the simulation
5. Collect and display the desired metrics

In [1]:
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    # begin by defining the simulation timeline with the correct simulation time
    tl = Timeline(sim_time * 1e9)
    tl.seed(0)
    
    # Here, we create nodes for the network (QKD nodes for key distribution)
    # stack_size=1 indicates that only the BB84 protocol should be included
    n1 = QKDNode("n1", tl, stack_size=1)
    n2 = QKDNode("n2", tl, stack_size=1)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
    
    # connect the nodes and set parameters for the fibers
    # note that channels are one-way
    # construct a classical communication channel
    # (with arguments for the channel name, timeline, and length (in m))
    cc0 = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc1 = ClassicalChannel("cc_n2_n1", tl, distance=1e3)
    cc0.set_ends(n1, n2.name)
    cc1.set_ends(n2, n1.name)
    qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc0.set_ends(n1, n2.name)
    qc1.set_ends(n2, n1.name)
    
    # instantiate our written keysize protocol
    km1 = KeyManager(tl, keysize, 25)
    km1.lower_protocols.append(n1.protocol_stack[0])
    n1.protocol_stack[0].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 25)
    km2.lower_protocols.append(n2.protocol_stack[0])
    n2.protocol_stack[0].upper_protocols.append(km2)
    
    # start simulation and record timing
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    # display our collected metrics
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ms)")
    plt.ylabel("Number of Completed Keys")
    plt.show()
    
    print("key error rates:")
    for i, e in enumerate(n1.protocol_stack[0].error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

### Running the Simulation

All that is left is to run the simulation with user input. (maximum execution time: ~5 sec)

Parameters:

    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)

In [5]:
# Create and run the simulation
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>

Due to the imperfect polarization fidelity specified for the optical fiber, we observe that most (if not all) of the completed keys have errors that render them unusable. For this reason, error correction protocols (such as cascade, which is included in SeQUeNCe and shown in the next example) must also be used.

## Example 2: Adding Cascade

This example is simular to the first example, with slight alterations to allow for

- Instatiation of the cascade error correction protocol on the qkd nodes
- Differences in the cascade `push`/`pop` interface compared to BB84

while the network topology remains unchanged.

In [6]:
from sequence.qkd.cascade import pair_cascade_protocols

class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) # interface for cascade to generate keys
            
    def pop(self, key): # interface for cascade to return generated keys
        self.keys.append(key)
        self.times.append(self.timeline.now() * 1e-9)
        
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    # begin by defining the simulation timeline with the correct simulation time
    tl = Timeline(sim_time * 1e9)
    tl.seed(0)
    
    # Here, we create nodes for the network (QKD nodes for key distribution)
    n1 = QKDNode("n1", tl)
    n2 = QKDNode("n2", tl)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
    pair_cascade_protocols(n1.protocol_stack[1], n2.protocol_stack[1])
    
    # connect the nodes and set parameters for the fibers
    cc0 = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc1 = ClassicalChannel("cc_n2_n1", tl, distance=1e3)
    cc0.set_ends(n1, n2.name)
    cc1.set_ends(n2, n1.name)
    qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=1e-5, distance=1e3,
                         polarization_fidelity=0.97)
    qc0.set_ends(n1, n2.name)
    qc1.set_ends(n2, n1.name)
    
    # instantiate our written keysize protocol
    km1 = KeyManager(tl, keysize, 10)
    km1.lower_protocols.append(n1.protocol_stack[1])
    n1.protocol_stack[1].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 10)
    km2.lower_protocols.append(n2.protocol_stack[1])
    n2.protocol_stack[1].upper_protocols.append(km2)
    
    # start simulation and record timing
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    # display our collected metrics
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ms)")
    plt.ylabel("Number of Completed Keys")
    plt.show()
    
    error_rates = []
    for i, key in enumerate(km1.keys):
        counter = 0
        diff = key ^ km2.keys[i]
        for j in range(km1.keysize):
            counter += (diff >> j) & 1
        error_rates.append(counter)

    print("key error rates:")
    for i, e in enumerate(error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

### Running the Simulation

We can now run the cascade simulation with user input. Note that the extra steps required by the cascade protocol may cause the simulation to run much longer than the example with only BB84.

Parameters:

    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    
The maximum execution time (`sim_time=1000`, `keysize=512`) is around 60 seconds.

In [7]:
# Create and run the simulation
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>

### Results

The implementation of the cascade protocol found within SeQUeNCe relies on the creation of a large sequence of bits, from which exerpts are used to create individual keys. Due to this behavior, keys are generated in large numbers in regularly spaced "batches". Also note that after application of error correction, the error rates for all keys are now at 0%.